# Phase 3: Heuristic Data Cleaning ("The Scrub")

**Objective:** To refine the raw dataset of potential refactoring candidates by algorithmically removing "false positives" and irrelevant noise.

**Methodology:**
Automated mining often captures commits that match keywords but do not represent valid refactoring data points for qualitative analysis. To address this, we apply a series of **Heuristic Filters** (a "Scrub") to remove four common categories of noise:

1.  **Merge Commits:** Commits generated by git merges (e.g., "Merge branch 'refactor-ui'") often contain keywords in the default message but do not represent a unique atomic change.
2.  **Documentation & Config:** Commits that modify *only* non-functional files (e.g., `README.md`, `.gitignore`, `LICENSE`) are excluded.
3.  **Asset Dumps:** In Unity development, importing 3D models or baking lighting often modifies dozens of files without touching source code. We define a "Dump" as a commit touching >50 files with **zero** C# script modifications.
4.  **Meta-Only Noise:** Commits that modify only `.meta` files (Unity's internal serialization) without accompanying asset or script changes are often unreadable editor artifacts and are excluded.

In [ ]:
import pandas as pd
import glob
import os
from datetime import datetime
from google.colab import drive

# --- CONFIGURATION ---
BASE_FOLDER = "/content/drive/My Drive/VR_Refactoring_Study"

# 1. Force Mount & Debug
# We check connection before attempting to load files
if not os.path.exists('/content/drive'):
    print("🔌 Mounting Google Drive...")
    drive.mount('/content/drive')
else:
    print("✅ Drive is already mounted.")

# 2. Robust Loader: Find the latest candidates file
def get_latest_candidates(folder):
    """
    Locates the most recently generated 'refactoring_candidates' CSV.
    """
    print(f"\n📂 Scanning directory: {folder}...")

    if not os.path.exists(folder):
        raise FileNotFoundError(f"❌ Error: The folder '{folder}' does not exist.")

    search_pattern = os.path.join(folder, "refactoring_candidates_*.csv")
    files = sorted(glob.glob(search_pattern))

    if not files:
        # Debugging aid if file is missing
        print("   ⚠️ No files found matching pattern.")
        print(f"   Contents of folder: {os.listdir(folder)}")
        raise FileNotFoundError(f"❌ No candidate files found. Did Phase 2 run successfully?")

    latest_file = files[-1]
    print(f"   Found {len(files)} candidate file(s).")
    print(f"✅ Selected latest dataset: {os.path.basename(latest_file)}")
    return latest_file

# Load Data
try:
    input_file = get_latest_candidates(BASE_FOLDER)
    df = pd.read_csv(input_file)
    initial_count = len(df)
    print(f"\n📊 Dataset Loaded Successfully.")
    print(f"   Total Raw Candidates: {initial_count}")
    print(f"   Columns: {list(df.columns)}")
except Exception as e:
    print(f"❌ Critical Error Loading Data: {e}")

🔌 Mounting Google Drive...
Mounted at /content/drive

📂 Scanning directory: /content/drive/My Drive/VR_Refactoring_Study...
   Found 1 candidate file(s).
✅ Selected latest dataset: refactoring_candidates_20260201.csv

📊 Dataset Loaded Successfully.
   Total Raw Candidates: 10745
   Columns: ['project_name', 'project_type', 'commit_hash', 'date', 'author', 'message', 'files_changed', 'refactor_category', 'repo_url']


In [ ]:
# @title Apply Heuristic Filters (Refactoring Focused)

def is_garbage(row):
    """
    Evaluates a commit against exclusion rules.
    Returns: (True, Reason) if garbage, (False, None) if valid.
    """
    msg = str(row['message']).lower()
    files_str = str(row['files_changed'])

    if files_str == "nan" or not files_str:
        files = []
    else:
        files = files_str.split('; ')

    # RULE 1: Merge Commits (Not a Refactor)
    if msg.startswith("merge branch") or msg.startswith("merge pull request"):
        return True, "Merge Commit"

    # RULE 2: Documentation & Config Only (Not a Refactor)
    doc_extensions = ['.md', '.txt', '.gitignore', '.license', '.yml', '.json', '.csproj', '.sln']
    if files and all(any(f.lower().endswith(ext) for ext in doc_extensions) for f in files):
        return True, "Docs/Config Only"

    # RULE 3: Massive Asset Dumps (Imports)
    # If >50 files change and ZERO are scripts/scenes/prefabs, it's likely a texture pack import.
    # We allow .unity/.prefab here because massive hierarchy changes ARE refactors.
    structure_ext = ['.cs', '.unity', '.prefab', '.shader']
    if len(files) > 50 and not any(any(f.lower().endswith(ext) for ext in structure_ext) for f in files):
        return True, "Massive Asset Dump (No Structure)"

    # RULE 4: Meta-Files Only (Reserialization)
    if files and all(f.lower().endswith('.meta') for f in files):
        return True, "Meta Files Only"

    # RULE 6: Lightmap / GI Cache (Data Re-bake, not Refactor)
    # LightingData.asset is just baked texture data.
    if any("lightingdata" in f.lower() or "lightmap-" in f.lower() or "gi-cache" in f.lower() for f in files):
        # Unless they also changed code/structure, it's just a bake
        if not any(any(f.lower().endswith(ext) for ext in structure_ext) for f in files):
            return True, "Lightmap/GI Rebuild"

    # RULE 7: External Plugin Noise (Dependency Update)
    # Changes in generic library folders are usually not user-refactors
    plugin_paths = ["library/", "packages/", "assets/plugins/", "assets/oculus/", "assets/steamvr/"]
    if files and all(any(p in f.lower() for p in plugin_paths) for f in files):
        return True, "External Plugin/Library Update"

    # RULE 9: Binary Assets Only (Content Update)
    # If the commit ONLY touches images/audio/models, it's an art update, not a refactor.
    binary_ext = ['.png','.jpg','.tga','.psd','.fbx','.obj','.wav','.mp3','.ogg','.anim','.controller']
    if files and all(any(f.lower().endswith(ext) for ext in binary_ext) for f in files):
        return True, "Binary Content Only"

    # RULE 10: Auto-Renamed GUIDs (Internal Unity Noise)
    # e.g. foo.mat and foo.mat.meta changing together without logic changes
    if files and all('.meta' in f.lower() or any(f.lower().endswith(ext) for ext in ['.mat','.anim']) for f in files):
         return True, "Auto Asset/Meta Regeneration"

    #RULE 11: Unrealistically Large Commit (Even with Code) ---
    # Commits > 50 are likely namespace renames, folder moves, or migrations.
    if len(files) > 50:
        return True, "Unrealistically Large Commit (>300 files)"

    return False, None

# --- EXECUTION ---
if 'df' in locals():
    print(f"🧹 Starting Refactoring Scrub on {initial_count} rows...")

    scrub_results = df.apply(is_garbage, axis=1)
    df['is_garbage'], df['removal_reason'] = zip(*scrub_results)

    df_clean = df[df['is_garbage'] == False].drop(columns=['is_garbage', 'removal_reason'])
    df_garbage = df[df['is_garbage'] == True]

    # --- REPORTING ---
    removed_count = len(df_garbage)
    final_count = len(df_clean)
    removal_rate = (removed_count/initial_count)*100

    print("\n" + "="*60)
    print(f"✅ SCRUB COMPLETE")
    print(f"🗑️  Removed: {removed_count} ({removal_rate:.1f}%)")
    print(f"✨ Remaining Refactoring Candidates: {final_count}")
    print("="*60)

    print("\n🔍 Removal Reasons Breakdown:")
    print(df_garbage['removal_reason'].value_counts())

    # --- SAVING ---
    timestamp = datetime.now().strftime("%Y%m%d")

    # 1. Save Cleaned Data
    clean_output = os.path.join(BASE_FOLDER, f"refactoring_candidates_CLEAN_{timestamp}.csv")
    df_clean.to_csv(clean_output, index=False)
    print(f"\n💾 Cleaned dataset saved to:\n   {clean_output}")

    # 2. Save Garbage Log (Audit Trail)
    garbage_output = os.path.join(BASE_FOLDER, f"garbage_log_{timestamp}.csv")
    df_garbage.to_csv(garbage_output, index=False)
    print(f"📝 Garbage/Audit log saved to:\n   {garbage_output}")

else:
    print("⚠️ DataFrame not found.")

🧹 Starting Refactoring Scrub on 10745 rows...

✅ SCRUB COMPLETE
🗑️  Removed: 1689 (15.7%)
✨ Remaining Refactoring Candidates: 9056

🔍 Removal Reasons Breakdown:
removal_reason
Unrealistically Large Commit (>300 files)    660
Merge Commit                                 510
Docs/Config Only                             321
Meta Files Only                               77
Massive Asset Dump (No Structure)             52
Auto Asset/Meta Regeneration                  43
Binary Content Only                           22
Lightmap/GI Rebuild                            4
Name: count, dtype: int64

💾 Cleaned dataset saved to:
   /content/drive/My Drive/VR_Refactoring_Study/refactoring_candidates_CLEAN_20260201.csv
📝 Garbage/Audit log saved to:
   /content/drive/My Drive/VR_Refactoring_Study/garbage_log_20260201.csv
